In [126]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import joblib
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [127]:
train_df =  pd.read_csv('n_train.csv', parse_dates=["startdate"])
test_df  =  pd.read_csv('n_test.csv', parse_dates=["startdate"])

In [128]:
tmp = pd.read_csv('sub_train.csv')
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'dayofyear', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
best_cols = [c for c in tmp.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(best_cols))

Main features: 54


In [129]:
best_cols

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'contest-prwtr-eatm-14d__prwtr',
 'contest-pres-sfc-gauss-14d__pres',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-9

In [130]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'dayofyear', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 246


In [131]:
train_df.groupby(['lat', 'lon'])[target[0]].mean().reset_index()

,lat,lon,contest-tmp2m-14d__tmp2m
0,0.000000,0.833333,23.474873
1,0.000000,0.866667,23.427205
2,0.045455,0.833333,23.170019
3,0.045455,0.866667,22.525948
4,0.045455,0.900000,22.475270
...,...,...,...
509,1.000000,0.733333,5.182245
510,1.000000,0.766667,5.431043
511,1.000000,0.800000,4.736491
512,1.000000,0.833333,4.268409


In [132]:
len(train_df['lon'].unique())

31

In [133]:
train_df = train_df.sort_values(by=['lat', 'lon', 'startdate']).reset_index(drop=True)

In [134]:
test_df = test_df.sort_values(by=['lat', 'lon', 'startdate']).reset_index(drop=True)

In [135]:
locations_df = train_df.groupby(['lat', 'lon'])['loc_group'].mean().reset_index()
locations_df['loc_group'] = locations_df['loc_group'].astype(np.int64)
locations_df

,lat,lon,loc_group
0,0.000000,0.833333,0
1,0.000000,0.866667,1
2,0.045455,0.833333,2
3,0.045455,0.866667,3
4,0.045455,0.900000,4
...,...,...,...
509,1.000000,0.733333,509
510,1.000000,0.766667,510
511,1.000000,0.800000,511
512,1.000000,0.833333,512


In [136]:
lat_list = locations_df['lat'].unique().tolist()
lat_df = pd.DataFrame(columns = ['lat', 'lat_index'])
lat_df['lat'] = lat_list
lat_df['lat_index'] = range(27, 27 + len(lat_list))
lat_df

,lat,lat_index
0,0.000000,27
1,0.045455,28
2,0.090909,29
3,0.136364,30
4,0.181818,31
5,0.227273,32
6,0.272727,33
7,0.318182,34
8,0.363636,35
9,0.409091,36


In [137]:
lon_list = locations_df['lon'].unique().tolist()
lon_df = pd.DataFrame(columns = ['lon', 'lon_index'])
lon_df['lon'] = lon_list
lon_df['lon_index'] = range(261, 261 + len(lon_list))
lon_df

,lon,lon_index
0,0.833333,261
1,0.866667,262
2,0.900000,263
3,0.800000,264
4,0.933333,265
5,0.666667,266
6,0.700000,267
7,0.733333,268
8,0.766667,269
9,0.966667,270


In [138]:
locations_df = locations_df.merge(lat_df, how='left', on='lat')
locations_df = locations_df.merge(lon_df, how='left', on='lon')
locations_df

,lat,lon,loc_group,lat_index,lon_index
0,0.000000,0.833333,0,27,261
1,0.000000,0.866667,1,27,262
2,0.045455,0.833333,2,28,261
3,0.045455,0.866667,3,28,262
4,0.045455,0.900000,4,28,263
...,...,...,...,...,...
509,1.000000,0.733333,509,49,268
510,1.000000,0.766667,510,49,269
511,1.000000,0.800000,511,49,264
512,1.000000,0.833333,512,49,261


In [253]:
region_df = pd.DataFrame(columns = ['lat_index', 'lon_index', 'region_id'])
region_df['lat_index'] = locations_df['lat_index']
region_df['lon_index'] = locations_df['lon_index']
region_df['region_id'] = [(region_df['lat_index'][i], region_df['lon_index'][i]) for i in range(len(region_df))]
region_df

,lat_index,lon_index,region_id
0,27,261,"(27, 261)"
1,27,262,"(27, 262)"
2,28,261,"(28, 261)"
3,28,262,"(28, 262)"
4,28,263,"(28, 263)"
...,...,...,...
509,49,268,"(49, 268)"
510,49,269,"(49, 269)"
511,49,264,"(49, 264)"
512,49,261,"(49, 261)"


In [140]:
train_df = train_df.merge(locations_df, on=['lat', 'lon'], how='left')
test_df = test_df.merge(locations_df, on=['lat', 'lon'], how='left')

In [215]:
train_features = ['startdate', 'lat_index', 'lon_index'] + best_cols + temporal_attrs
test_features = ['startdate', 'lat_index', 'lon_index'] + best_cols + temporal_attrs
X_train_df = train_df[train_features]
y_train_df = train_df[target[0]]
test_df = test_df[test_features]

In [274]:
X_test_df = test_df[test_features]

In [216]:
len(train_df['lat_index'].unique())

23

In [217]:
for c in (best_cols + temporal_attrs):
    if train_df[c].std() < 1:
        print(c, train_df[c].std())

year 0.6876273873198657
day_of_year_sin 0.707358050479475
day_of_year_cos 0.7068559814326024
month_sin 0.7058851123787788
month_cos 0.7083138293964283
season_sin 0.7075859251585764
season_cos 0.7066119810690936


In [265]:
class PreprocessingTemporalSpatialData:
    def __init__(self, data:np.array, locations: np.array, col_names, num_regions: int, num_features: int, max_sg: int = 5):
        self.data = data
        self.locations = locations
        self.regions = self.locations['region_id'].unique()
        self.latmin, self.latmax, self.lonmin, self.lonmax = self.locations['lat_index'].min(), self.locations['lat_index'].max(), \
            self.locations['lon_index'].min(), self.locations['lon_index'].max()
        self.all_region_ids = [(lat, lon) for lat in range(self.latmin, self.latmax + 1) for lon in
                               range(self.lonmin, self.lonmax + 1)]
        self.num_total_regions = len(set(self.all_region_ids))

        
        self.bin_width, self.bin_height = self.lonmax - self.lonmin + 1, self.latmax - self.latmin + 1
        self.col_names = col_names
        self.num_regions = num_regions
        self.num_features = num_features
        self.weather_features = 54
        self.cyclical_features = 10
        self.num_timesteps = self.data.reshape(self.num_regions, -1, self.num_features).shape[1]
        self.max_sg = max_sg
    
    def standard_and_scale_data(self, save=False):
        
        TRAIN_SPLIT = self.num_timesteps

        date = self.data[:, 0].reshape((-1, 1))
        lat_lon = self.data[:, 1:3]

        features = self.data[:, 3:-self.cyclical_features].astype(np.float64)
        self.data_mean = features.mean(axis=0)
        self.data_std = features.std(axis=0)
        features = ((features - self.data_mean) / self.data_std).astype(np.float64)

        cyclical_features = self.data[:, -self.cyclical_features:].astype(np.float64)
        

        all_features = np.hstack((features, cyclical_features))

        scaler = MinMaxScaler()
        all_features = scaler.fit_transform(all_features)
        
        temp_scaler = MinMaxScaler()
        temp_scaler.fit(np.array(features[:, 3].reshape(-1, 1)))

        if save:
            print("Save scaler:")
            np.save('./feature_means', self.data.mean)
            np.save('./feature_stds', self.data_std)
            joblib.dump(scaler, './all_feature_scaler.pkl')
            joblib.dump(temp_scaler, './temp_scaler.pkl')
        
        self.data = np.hstack((date, lat_lon, all_features))
        self.data = self.data.reshape(-1, self.num_features)
    
    def get_missing_regions(self):
        """ Find regions in lat-lon box which are not modelled geographic regions  """
        self.all_region_ids = [(lat, lon) for lat in range(self.latmin, self.latmax + 1) for lon in
                               range(self.lonmin, self.lonmax + 1)]
        self.num_total_regions = len(set(self.all_region_ids))
        self.missing_regions = list(set(self.all_region_ids) - set(list(self.regions)))
        self.missing_regions.sort()
    

    def process_datetime(self, dt_fmt: str = '%Y-%m-%d', datetimecol='startdate'):
        self.all_dates = np.unique(self.data["startdate"])
    
    def mask_missing_regions(self, mask_value=0):
      
        self.get_missing_regions()
        masked_rgn_lst = []
        for rgn in self.missing_regions:
            date_col = self.data.reshape(self.num_regions, -1, self.num_features)[0, :, 0].reshape(self.num_timesteps,
                                                                                                   1)  # take same dates
            lat_col = np.array([rgn[0]] * self.num_timesteps).reshape(self.num_timesteps,
                                                                      1)  # take lat of current region
            lon_col = np.array([rgn[1]] * self.num_timesteps).reshape(self.num_timesteps,
                                                                      1)  # take lon of current region
            feature_cols = np.array([mask_value] * self.num_timesteps * (self.weather_features)).reshape(
                self.num_timesteps, self.weather_features)  # mask weather features
            cyclical_cols = self.data.reshape(self.num_regions, -1, self.num_features)[0, :,
                            -self.cyclical_features:].reshape(self.num_timesteps,
                                                              self.cyclical_features)  # take time features
            masked_rgn = np.hstack((date_col, lat_col, lon_col, feature_cols, cyclical_cols))
            masked_rgn_lst.append(masked_rgn)
        masked_rgns = np.concatenate(masked_rgn_lst)
        self.masked_rgns_df = pd.DataFrame(masked_rgns)
        self.masked_rgns_df.columns = self.col_names
        self.masked_rgns_df['region_id'] = list(
            zip(self.masked_rgns_df['lat_index'].astype(int), self.masked_rgns_df['lon_index'].astype(int)))
        self.masked_rgns_df['model_region'] = False
    
    def convert_rgns_data_to_df(self):
        self.data = pd.DataFrame(self.data.reshape(-1, self.num_features))
        self.data.columns = self.col_names
        self.data['region_id'] = list(zip(self.data['lat_index'], self.data['lon_index']))
        self.data['model_region'] = True
    
    def join_masked_regions(self):
        self.data = pd.concat([self.data, self.masked_rgns_df])
        self.process_datetime()
        self.data = self.data.sort_values(by=['lat_index', 'lon_index', 'startdate']).reset_index(drop=True)
        assert self.num_total_regions == len(self.data['region_id'].unique())
    
    def get_global_region_tensor(self, save=False):
        spatial_tw_list = []
        for counter, time_window in enumerate(self.all_dates):
            mask = self.data['startdate'] == np.datetime64(time_window)
            pvt_current_time_window = np.flipud(np.array(self.data[mask]).reshape(self.bin_height, self.bin_width, -1))
            spatial_tw_list.append(pvt_current_time_window)
        self.global_region_tensor = np.stack(spatial_tw_list).reshape(-1, self.bin_height, self.bin_width,
                                                                      pvt_current_time_window.shape[2])
        self.global_region_tensor = self.global_region_tensor[:, :, :, :-2]

        npad = ((0, 0), (self.max_sg, self.max_sg), (self.max_sg, self.max_sg),
                (0, 0))  
        self.global_region_tensor = np.pad(self.global_region_tensor, pad_width=npad, mode='constant',
                                           constant_values=0)

        if save:
            print("Saving global spatial tensor \n")
            np.save('./global_region_tensor_scaled_sg' + str(self.max_sg),
                    self.global_region_tensor)
    
    def preprocess_pipeline(self):
        self.standard_and_scale_data(save=True)
        self.mask_missing_regions()
        self.convert_rgns_data_to_df()
        self.join_masked_regions()
        self.get_global_region_tensor(save=True)
    def get_result(self):
        return self.data

In [266]:
X_train_df[['lat_index', 'lon_index', 'startdate']]

,lat_index,lon_index,startdate
0,27,261,2014-09-01
1,27,261,2014-09-02
2,27,261,2014-09-03
3,27,261,2014-09-04
4,27,261,2014-09-05
...,...,...,...
375729,49,262,2016-08-27
375730,49,262,2016-08-28
375731,49,262,2016-08-29
375732,49,262,2016-08-30


In [267]:
len(X_train_df['lon_index'].unique())

31

In [268]:
col_names = train_features
y = PreprocessingTemporalSpatialData(X_train_df.values, region_df, col_names, num_regions=514, num_features=len(col_names), max_sg=5)
y.preprocess_pipeline()

Save scaler:
Saving global spatial tensor 



In [273]:
scaled_train_df = y.get_result()
np.save('scaled_train', scaled_train_df)

In [276]:
scaled_train_df.shape

(521203, 69)

In [275]:
col_names = test_features
y = PreprocessingTemporalSpatialData(X_test_df.values, region_df, col_names, num_regions=514, num_features=len(col_names), max_sg=5)
y.preprocess_pipeline()

scaled_test_df = y.get_result()
np.save('scaled_test', scaled_test_df)

Save scaler:
Saving global spatial tensor 

